In [ ]:
import keras
from keras import backend as K
from keras.layers import Dense, Input
import numpy as np
import tensorflow as tf
import math
import h5py
import random
import time
import pandas as pd
from context import * # imports MDN
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# Set up environment.
# Only for GPU use:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from keras import backend as K
K.set_session(sess)

In [ ]:
### Loading from the saved datasets.
with np.load('../datasets/tiny_performance_datasets.npz') as loaded:
    loaded_raw = loaded['raw_perfs']
    loaded_diff = loaded['diff_perfs']
    
print("Loaded perfs:", len(loaded_raw), "and", len(loaded_diff))
print("Num touches:", np.sum([len(l) for l in loaded_raw]))

corpus = []

for l in loaded_raw:
    corpus.append(l[:,:])
    
    
# Plot a bit of the data to have a look:
robojam.plot_2D(robojam.perf_array_to_df(random.choice(corpus)))

In [ ]:
# Model Hyperparameters:
HIDDEN_UNITS = 128
N_LAYERS = 2
NUMBER_MIXTURES = 5
TIME_DIST = False

# Data Hyperparameters:
SEQ_LEN = 30
SEQ_STEP = 2

# Training Hyperparameters:
BATCH_SIZE = 64
EPOCHS = 30
VAL_SPLIT = 0.10

# Set random seed for reproducibility
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)

# Restrict corpus to sequences longer than the corpus.
corpus = [l for l in corpus if len(l) > SEQ_LEN+1]
print("Corpus Examples:", len(corpus))

# Prepare training data as X and Y.
slices = []
for seq in corpus:
    slices += robojam.slice_sequence_examples(seq, SEQ_LEN+1, step_size=SEQ_STEP)

if TIME_DIST:
    X, y = robojam.seq_to_overlapping_format(slices)
else:
    X, y = robojam.seq_to_singleton_format(slices)

X = np.array(X) * robojam.SCALE_FACTOR
y = np.array(y) * robojam.SCALE_FACTOR

print("Number of training examples:")
print("X:", X.shape)
print("y:", y.shape)

In [ ]:
# Setup Training Model
model = robojam.build_robojam_model(seq_len=SEQ_LEN, hidden_units=HIDDEN_UNITS, num_mixtures=NUMBER_MIXTURES, layers=2, time_dist=TIME_DIST, inference=False, compile_model=True, print_summary=True, predict_moving=True)

In [ ]:
# Setup callbacks
model_path = "robojam-4D" + "-layers" + str(N_LAYERS) + "-units" + str(HIDDEN_UNITS) + "-mixtures" + str(NUMBER_MIXTURES) + "-scale" + str(robojam.SCALE_FACTOR)
filepath = model_path + "-E{epoch:02d}-VL{val_loss:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
terminateOnNaN = keras.callbacks.TerminateOnNaN()
tboard = keras.callbacks.TensorBoard(log_dir='../logs/'+model_path, update_freq='epoch')
callbacks = [checkpoint, terminateOnNaN, tboard]
# Train
history = model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VAL_SPLIT, callbacks=callbacks)
#history = model.fit_generator(generator, steps_per_epoch=300, epochs=100, verbose=1, initial_epoch=0)

# Save final Model
# model.save('robojam-model-final.hdf5')  # creates a HDF5 file of the model

# Plot the loss


In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
model_file = "robojam-tinyperf-layers2-units32-mixtures5-scale10-E03-VL-0.75.hdf5"

decoder = robojam.build_robojam_model(seq_len=SEQ_LEN, hidden_units=HIDDEN_UNITS, num_mixtures=NUMBER_MIXTURES, layers=N_LAYERS, time_dist=TIME_DIST, inference=True, compile_model=False, print_summary=True,  predict_moving=True)
#decoder.load_weights("../robojam-model-layers2-units512-mixtures5-scale10-E13-VL-5.79.hdf5")
decoder.load_weights(model_file)

In [ ]:
length = 100
# t = random.randint(0,len(microjam_corpus)-length)
t = 1000
mixture_temp = 1.5
sigma_temp = 0.01
ex =  random.choice(corpus) #microjam_corpus[t:t+length]  #sequences[600] 

decoder.reset_states()
p = robojam.condition_and_generate(decoder, ex, NUMBER_MIXTURES, temp=mixture_temp, sigma_temp=sigma_temp,  predict_moving=True)
robojam.plot_double_2d(robojam.perf_array_to_df(ex), robojam.perf_array_to_df(p), name="robojam-perf-cond", saving=False)

#plot_double_3d(robojam.perf_array_to_df(ex), robojam.perf_array_to_df(p), name="robojam-perf-cond-3d", saving=False)

In [ ]:
decoder.reset_states()
mixture_temp = 1.5
sigma_temp = 0.0
t = robojam.random_touch(with_moving=True)
p = robojam.generate_random_tiny_performance(decoder, NUMBER_MIXTURES, t, temp=mixture_temp, sigma_temp=sigma_temp,  predict_moving=True)
robojam.plot_2D(robojam.perf_array_to_df(p))

#plot_3D(robojam.perf_array_to_df(p))